![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/9.Chunk_Key_Phrase_Extraction.ipynb)

# Chunk Key Phrase Extraction

In this notebook, you will find how to get chunk key phrases using `ChunkKeyPhraseExtraction` that leverages Sentence BERT embeddings to select keywords and key phrases that are most similar to a document. This annotator can be fed by either the output of NER model, NGramGenerator or YAKE, and could be used to generate similarity scores for each NER chunk that is coming out of any (clinical) NER model. That is, you can now sort your clinical entities by the importance of them with respect to document or sentence that they live in. Additionally, you can also use this new annotator to grab new clinical chunks that are missed by a pretrained NER model as well as summarizing the whole document into a few important sentences or phrases.

In [ ]:
import sys
import json
import os
with open('license.json') as f:
    license_keys = json.load(f)
    
import os
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
import json
import os

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sparknlp_jsl
import sparknlp

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.5.0


**Lets start with creating a spark dataframe.**

In [4]:
text = """
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight 
years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior 
episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute 
hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week 
history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation, 
she was treated with a five-day course of amoxicillin for a respiratory tract infection. 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for 
HTG. She had been on dapagliflozin for six months at the time of presentation . Physical examination 
on presentation was significant for dry oral mucosa ; significantly, her abdominal examination was 
benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were: 
serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 
508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27. 
Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept
hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis, 
as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained 
six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21, 
serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L. 
The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was 
centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by 
lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion 
gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by 
her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the 
endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin 
lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors 
should be discontinued indefinitely . She had close follow-up with endocrinology post discharge.
""".strip().replace("\n", "")

empty_data = spark.createDataFrame([[""]]).toDF("text")
textDF = spark.createDataFrame([[text]]).toDF("text")

# with NGramGenerator
First, we will show you how to get chunk key phrases using N-Gram by feeding `ChunkKeyPhraseExtraction` with N-Gram output.

In [5]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentencer = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("tokens") \
    .setSplitChars(['\[','\]']) \

stop_words_cleaner = StopWordsCleaner.pretrained()\
    .setInputCols("tokens")\
    .setOutputCol("clean_tokens")\
    .setCaseSensitive(False)

ngram_generator = NGramGenerator()\
    .setInputCols(["clean_tokens"])\
    .setOutputCol("ngrams")\
    .setN(3)

ngram_key_phrase_extractor = ChunkKeyPhraseExtraction.pretrained()\
    .setTopN(10) \
    .setDivergence(0.4)\
    .setInputCols(["sentences", "ngrams"])\
    .setOutputCol("ngram_key_phrases")

ngram_pipeline = Pipeline(stages=[
    documenter, 
    sentencer, 
    tokenizer, 
    stop_words_cleaner,
    ngram_generator,
    ngram_key_phrase_extractor
])

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
sbert_jsl_medium_uncased download started this may take some time.
Approximate size to download 146.8 MB
[OK!]


In [6]:
ngram_results = ngram_pipeline.fit(empty_data).transform(textDF)

**Lets show N-Gram results.**

In [7]:
ngram_results.selectExpr("explode(ngrams) AS key_phrase_candidate").show(30,truncate=False)

+-------------------------------------------------------------------------------------+
|key_phrase_candidate                                                                 |
+-------------------------------------------------------------------------------------+
|{chunk, 2, 34, 28-year-old female history, {sentence -> 0, chunk -> 0}, []}          |
|{chunk, 14, 49, female history gestational, {sentence -> 0, chunk -> 1}, []}         |
|{chunk, 28, 58, history gestational diabetes, {sentence -> 0, chunk -> 2}, []}       |
|{chunk, 39, 67, gestational diabetes mellitus, {sentence -> 0, chunk -> 3}, []}      |
|{chunk, 51, 77, diabetes mellitus diagnosed, {sentence -> 0, chunk -> 4}, []}        |
|{chunk, 60, 89, mellitus diagnosed years, {sentence -> 0, chunk -> 5}, []}           |
|{chunk, 69, 95, diagnosed years prior, {sentence -> 0, chunk -> 6}, []}              |
|{chunk, 85, 111, years prior presentation, {sentence -> 0, chunk -> 7}, []}          |
|{chunk, 91, 126, prior presenta

**Check the key phrases from N-Gram results.**

In [8]:
ngram_results.selectExpr("explode(ngram_key_phrases) AS ngram_key_phrases").show(truncate=170)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                         ngram_key_phrases|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{chunk, 117, 144, subsequent type diabetes, {sentence -> 0, chunk -> 10, DocumentSimilarity -> 0.750370907052758, MMRScore -> 0.4502225621218913}, [0.27310055, -1.5514...|
|{chunk, 51, 77, diabetes mellitus diagnosed, {sentence -> 0, chunk -> 4, DocumentSimilarity -> 0.6953794678667229, MMRScore -> 0.08263045122125517}, [-0.41446576, -1.9...|
|{chunk, 186, 221, HTG-induced pancreatitis years, {sentence -> 0, chunk -> 19, DocumentSimilarity -> 0.6817062670747339, MMRScore -> 0

**Show the selected key phrases, the cosine similarity to the document, the Maximal Marginal Relevance score and the sentence they where key phrase was found in.**

In [9]:
ngram_results.select(F.explode(F.arrays_zip("ngram_key_phrases.result","ngram_key_phrases.metadata")).alias("cols"))\
              .select(F.expr("cols['0']").alias("key_phrase"),
                      F.expr("cols['1']['DocumentSimilarity']").alias("DocumentSimilarity"),
                      F.expr("cols['1']['MMRScore']").alias("MMRScore"),
                      F.expr("cols['1']['sentence']").alias("sentence")).show(truncate=False)

+--------------------------------+-------------------+-------------------+--------+
|key_phrase                      |DocumentSimilarity |MMRScore           |sentence|
+--------------------------------+-------------------+-------------------+--------+
|subsequent type diabetes        |0.750370907052758  |0.4502225621218913 |0       |
|diabetes mellitus diagnosed     |0.6953794678667229 |0.08263045122125517|0       |
|HTG-induced pancreatitis years  |0.6817062670747339 |0.09449722527630255|0       |
|hepatitis obesity               |0.6666055227936242 |0.10480015943732174|0       |
|mellitus diagnosed years        |0.6389213843915571 |0.09924538733998584|0       |
|vomiting                        |0.5824238685505062 |0.13604066449141936|0       |
|admitted starvation ketosis     |0.5789874663466789 |0.09807403604602011|0       |
|five-day amoxicillin respiratory|0.5330654430401827 |0.11179572344935851|0       |
|33.5 kg/m2                      |0.46256537062506975|0.08639883387550154|0 

# with NER Model

Now we will show how to get key phrases from NER chunks by feeding `ChunkKeyPhraseExtraction` with the output of `NerConverter`.

In [10]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentencer = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("tokens") \
    .setSplitChars(['\[','\]']) \

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models") \
    .setInputCols(["document", "tokens"]) \
    .setOutputCol("embeddings")

ner_tagger = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_tags")

ner_converter = NerConverter()\
    .setInputCols("sentences", "tokens", "ner_tags")\
    .setOutputCol("ner_chunks")

ner_key_phrase_extractor = ChunkKeyPhraseExtraction.pretrained()\
    .setTopN(10) \
    .setDivergence(0.4)\
    .setInputCols(["sentences", "ner_chunks"])\
    .setOutputCol("ner_key_phrases")

ner_pipeline = Pipeline(stages=[
    documenter, 
    sentencer, 
    tokenizer, 
    embeddings, 
    ner_tagger, 
    ner_converter, 
    ner_key_phrase_extractor
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
sbert_jsl_medium_uncased download started this may take some time.
Approximate size to download 146.8 MB
[OK!]


In [11]:
ner_results = ner_pipeline.fit(empty_data).transform(textDF)

In [12]:
# ner_chunk results

ner_results.select(F.explode(F.arrays_zip("ner_chunks.result","ner_chunks.metadata")).alias("cols"))\
           .select(F.expr("cols['0']").alias("ner_chunk"),
                   F.expr("cols['1']['entity']").alias("label")).show(50, truncate=False)

+-----------------------------+-------------------------+
|ner_chunk                    |label                    |
+-----------------------------+-------------------------+
|28-year-old                  |Age                      |
|female                       |Gender                   |
|gestational diabetes mellitus|Diabetes                 |
|eight years prior            |RelativeDate             |
|type two diabetes mellitus   |Diabetes                 |
|T2DM                         |Diabetes                 |
|HTG-induced pancreatitis     |Disease_Syndrome_Disorder|
|three years prior            |RelativeDate             |
|acute                        |Modifier                 |
|hepatitis                    |Communicable_Disease     |
|obesity                      |Obesity                  |
|body mass index              |Symptom                  |
|33.5 kg/m2                   |Weight                   |
|one-week                     |Duration                 |
|polyuria     

**Show the key phrase results and scores we got using NER chunks.**

In [13]:
ner_results.select(F.explode(F.arrays_zip("ner_key_phrases.result","ner_key_phrases.metadata")).alias("cols"))\
           .select(F.expr("cols['0']").alias("key_phrase"),
                   F.expr("cols['1']['entity']").alias("label"),
                   F.expr("cols['1']['DocumentSimilarity']").alias("DocumentSimilarity"),
                   F.expr("cols['1']['MMRScore']").alias("MMRScore"),
                   F.expr("cols['1']['sentence']").alias("sentence")).show(truncate=False)

+--------------------------+-------------------------+-------------------+-------------------+--------+
|key_phrase                |label                    |DocumentSimilarity |MMRScore           |sentence|
+--------------------------+-------------------------+-------------------+-------------------+--------+
|type two diabetes mellitus|Diabetes                 |0.7639752178409145 |0.4583851489191373 |0       |
|HTG-induced pancreatitis  |Disease_Syndrome_Disorder|0.6693321970300268 |0.10416350494181925|0       |
|hepatitis                 |Communicable_Disease     |0.6052964221031224 |0.0684493996556495 |0       |
|vomiting                  |Symptom                  |0.5824236751787064 |0.14864177343228657|0       |
|starvation ketosis        |Disease_Syndrome_Disorder|0.5540198650204701 |0.0901427854508885 |9       |
|lipemia                   |Disease_Syndrome_Disorder|0.5382694886607319 |0.04719087080054196|8       |
|obesity                   |Obesity                  |0.50024988

# with NGramGenerator and NER Model

We can also get key phrases from merging N-Gram and NER chunks.

In [14]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentencer = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("tokens") \
    .setSplitChars(['\[','\]']) \

stop_words_cleaner = StopWordsCleaner.pretrained()\
    .setInputCols("tokens")\
    .setOutputCol("clean_tokens")\
    .setCaseSensitive(False)

ngram_generator = NGramGenerator()\
    .setInputCols(["clean_tokens"])\
    .setOutputCol("ngrams")\
    .setN(3)
        
embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models") \
    .setInputCols(["document", "tokens"]) \
    .setOutputCol("embeddings")

ner_tagger = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_tags")

ner_converter = NerConverter()\
    .setInputCols("sentences", "tokens", "ner_tags")\
    .setOutputCol("ner_chunks")

chunk_merger = ChunkMergeApproach()\
    .setInputCols("ngrams", "ner_chunks")\
    .setOutputCol("merged_chunks")\
    .setMergeOverlapping(False)

ngram_ner_key_phrase_extractor = ChunkKeyPhraseExtraction.pretrained()\
    .setTopN(10) \
    .setDivergence(0.4)\
    .setInputCols(["sentences", "merged_chunks"])\
    .setOutputCol("key_phrases")

ngram_ner_pipeline = Pipeline(stages=[
    documenter, 
    sentencer, 
    tokenizer, 
    stop_words_cleaner,
    ngram_generator,
    embeddings, 
    ner_tagger, 
    ner_converter, 
    chunk_merger,
    ngram_ner_key_phrase_extractor
])

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
sbert_jsl_medium_uncased download started this may take some time.
Approximate size to download 146.8 MB
[OK!]


In [15]:
ngram_ner_results = ngram_ner_pipeline.fit(empty_data).transform(textDF)

**Show the merged key phrase candidate results. `UNK` ones from NGramGenerator and the others from `ner_jsl` model.**

In [16]:
ngram_ner_results.selectExpr("explode(merged_chunks) AS key_phrase_candidate").show(30,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------+
|key_phrase_candidate                                                                                                                    |
+----------------------------------------------------------------------------------------------------------------------------------------+
|{chunk, 2, 34, 28-year-old female history, {entity -> UNK, chunk -> 0, sentence -> 0}, []}                                              |
|{chunk, 2, 12, 28-year-old, {entity -> Age, confidence -> 1.0, chunk -> 1, sentence -> 0}, []}                                          |
|{chunk, 14, 49, female history gestational, {entity -> UNK, chunk -> 2, sentence -> 0}, []}                                             |
|{chunk, 14, 19, female, {entity -> Gender, confidence -> 0.9985, chunk -> 3, sentence -> 0}, []}                                        |
|{chunk, 28, 58, history ge

In [17]:
# NER chunk results

ngram_ner_results.select(F.explode(F.arrays_zip("merged_chunks.result","merged_chunks.metadata")).alias("cols"))\
                 .select(F.expr("cols['0']").alias("key_phrase_candidate"),
                         F.expr("cols['1']['entity']").alias("label")).filter("label != 'UNK'").show(50, truncate=False)

+-----------------------------+-------------------------+
|key_phrase_candidate         |label                    |
+-----------------------------+-------------------------+
|28-year-old                  |Age                      |
|female                       |Gender                   |
|gestational diabetes mellitus|Diabetes                 |
|eight years prior            |RelativeDate             |
|type two diabetes mellitus   |Diabetes                 |
|T2DM                         |Diabetes                 |
|HTG-induced pancreatitis     |Disease_Syndrome_Disorder|
|three years prior            |RelativeDate             |
|acute                        |Modifier                 |
|hepatitis                    |Communicable_Disease     |
|obesity                      |Obesity                  |
|body mass index              |Symptom                  |
|33.5 kg/m2                   |Weight                   |
|one-week                     |Duration                 |
|polyuria     

In [18]:
# ngram results

ngram_ner_results.select(F.explode(F.arrays_zip("merged_chunks.result","merged_chunks.metadata")).alias("cols"))\
                 .select(F.expr("cols['0']").alias("key_phrase_candidate"),
                         F.expr("cols['1']['entity']").alias("label")).filter("label == 'UNK'").show(50, truncate=False)

+--------------------------------+-----+
|key_phrase_candidate            |label|
+--------------------------------+-----+
|28-year-old female history      |UNK  |
|female history gestational      |UNK  |
|history gestational diabetes    |UNK  |
|gestational diabetes mellitus   |UNK  |
|diabetes mellitus diagnosed     |UNK  |
|mellitus diagnosed years        |UNK  |
|diagnosed years prior           |UNK  |
|years prior presentation        |UNK  |
|prior presentation subsequent   |UNK  |
|presentation subsequent type    |UNK  |
|subsequent type diabetes        |UNK  |
|type diabetes mellitus          |UNK  |
|diabetes mellitus (             |UNK  |
|mellitus ( T2DM                 |UNK  |
|( T2DM ),                       |UNK  |
|T2DM ), prior                   |UNK  |
|), prior episode                |UNK  |
|prior episode HTG-induced       |UNK  |
|episode HTG-induced pancreatitis|UNK  |
|HTG-induced pancreatitis years  |UNK  |
|pancreatitis years prior        |UNK  |
|years prior pre

In [19]:
# merged (NER chunk + ngram) results

ngram_ner_results.select(F.explode(F.arrays_zip("merged_chunks.result","merged_chunks.metadata")).alias("cols"))\
                 .select(F.expr("cols['0']").alias("key_phrase_candidate"),
                         F.expr("cols['1']['entity']").alias("label")).show(50, truncate=False)

+--------------------------------+-------------------------+
|key_phrase_candidate            |label                    |
+--------------------------------+-------------------------+
|28-year-old female history      |UNK                      |
|28-year-old                     |Age                      |
|female history gestational      |UNK                      |
|female                          |Gender                   |
|history gestational diabetes    |UNK                      |
|gestational diabetes mellitus   |UNK                      |
|gestational diabetes mellitus   |Diabetes                 |
|diabetes mellitus diagnosed     |UNK                      |
|mellitus diagnosed years        |UNK                      |
|diagnosed years prior           |UNK                      |
|eight years prior               |RelativeDate             |
|years prior presentation        |UNK                      |
|prior presentation subsequent   |UNK                      |
|presentation subsequent

**Show the key phrase candidates and their source (NER or NGramGenerator).**

In [20]:
ngram_ner_results.selectExpr("explode(merged_chunks) AS key_phrase_candidate")\
                 .selectExpr("key_phrase_candidate.result AS key_phrase_candidate",
                             "IF(key_phrase_candidate.metadata.entity = 'UNK', 'ngram', 'NER') AS source",
                             "key_phrase_candidate.metadata.sentence")\
                 .show(50, truncate=False)

+--------------------------------+------+--------+
|key_phrase_candidate            |source|sentence|
+--------------------------------+------+--------+
|28-year-old female history      |ngram |0       |
|28-year-old                     |NER   |0       |
|female history gestational      |ngram |0       |
|female                          |NER   |0       |
|history gestational diabetes    |ngram |0       |
|gestational diabetes mellitus   |ngram |0       |
|gestational diabetes mellitus   |NER   |0       |
|diabetes mellitus diagnosed     |ngram |0       |
|mellitus diagnosed years        |ngram |0       |
|diagnosed years prior           |ngram |0       |
|eight years prior               |NER   |0       |
|years prior presentation        |ngram |0       |
|prior presentation subsequent   |ngram |0       |
|presentation subsequent type    |ngram |0       |
|subsequent type diabetes        |ngram |0       |
|type diabetes mellitus          |ngram |0       |
|type two diabetes mellitus    

**Show the extracted key phrases and their scores.**

In [21]:
ngram_ner_results.select(F.explode(F.arrays_zip("key_phrases.result","key_phrases.metadata")).alias("cols"))\
                 .select(F.expr("cols['0']").alias("key_phrase"),
                         F.expr("cols['1']['entity']").alias("label"),
                         F.expr("cols['1']['DocumentSimilarity']").alias("DocumentSimilarity"),
                         F.expr("cols['1']['MMRScore']").alias("MMRScore"),
                         F.expr("cols['1']['sentence']").alias("sentence")).show(truncate=False)

+--------------------------------+--------+-------------------+-------------------+--------+
|key_phrase                      |label   |DocumentSimilarity |MMRScore           |sentence|
+--------------------------------+--------+-------------------+-------------------+--------+
|type two diabetes mellitus      |Diabetes|0.7639752178409145 |0.4583851489191373 |0       |
|subsequent type diabetes        |UNK     |0.7503709550950611 |0.0829824679140303 |0       |
|HTG-induced pancreatitis years  |UNK     |0.6817062670747339 |0.11246277713293629|0       |
|hepatitis obesity               |UNK     |0.6666055227936242 |0.11770535787141301|0       |
|mellitus diagnosed years        |UNK     |0.6389213593287539 |0.0812948472932965 |0       |
|history gestational diabetes    |UNK     |0.6219875504416315 |0.09501042715667779|0       |
|vomiting                        |UNK     |0.5824236751787064 |0.14864177343228657|0       |
|admitted starvation ketosis     |UNK     |0.5789874663466789 |0.12008

**Show the extracted key phrases and their sources.**

In [22]:
ngram_ner_results.selectExpr("explode(key_phrases) AS key_phrase")\
                 .selectExpr(
                     "SUBSTRING(key_phrase.result, 0, 40) as key_phrase",
                     "IF(key_phrase.metadata.entity = 'UNK', 'ngrams', 'NER') AS source",
                     "key_phrase.metadata.DocumentSimilarity",
                     "key_phrase.metadata.MMRScore",
                     "key_phrase.metadata.sentence")\
                 .show(truncate=False)

+--------------------------------+------+-------------------+-------------------+--------+
|key_phrase                      |source|DocumentSimilarity |MMRScore           |sentence|
+--------------------------------+------+-------------------+-------------------+--------+
|type two diabetes mellitus      |NER   |0.7639752178409145 |0.4583851489191373 |0       |
|subsequent type diabetes        |ngrams|0.7503709550950611 |0.0829824679140303 |0       |
|HTG-induced pancreatitis years  |ngrams|0.6817062670747339 |0.11246277713293629|0       |
|hepatitis obesity               |ngrams|0.6666055227936242 |0.11770535787141301|0       |
|mellitus diagnosed years        |ngrams|0.6389213593287539 |0.0812948472932965 |0       |
|history gestational diabetes    |ngrams|0.6219875504416315 |0.09501042715667779|0       |
|vomiting                        |ngrams|0.5824236751787064 |0.14864177343228657|0       |
|admitted starvation ketosis     |ngrams|0.5789874663466789 |0.12008074933190521|0       |

**Now we will change the default embeddings of `ChunkKeyPhraseExtraction` (`sbert_jsl_medium_uncased`) to `sbiobert_base_cased_mli` and see the results.**

In [23]:
ngram_ner_key_phrase_bio = ChunkKeyPhraseExtraction.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setTopN(10) \
    .setDivergence(0.4)\
    .setInputCols(["sentences", "merged_chunks"])\
    .setOutputCol("key_phrases")

ngram_ner_bio_pipeline = Pipeline(stages=[
    documenter, 
    sentencer, 
    tokenizer, 
    stop_words_cleaner,
    ngram_generator,
    embeddings, 
    ner_tagger, 
    ner_converter, 
    chunk_merger,
    ngram_ner_key_phrase_bio
])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [24]:
ngram_ner_bio_results = ngram_ner_bio_pipeline.fit(empty_data).transform(textDF)

In [25]:
# sbiobert_base_cased_mli

ngram_ner_bio_results.selectExpr("explode(key_phrases) AS key_phrase")\
                     .selectExpr(
                         "SUBSTRING(key_phrase.result, 0, 40) as key_phrase",
                         "IF(key_phrase.metadata.entity = 'UNK', 'ngrams', 'NER') AS source",
                         "key_phrase.metadata.DocumentSimilarity",
                         "key_phrase.metadata.MMRScore",
                         "key_phrase.metadata.sentence")\
                     .show(truncate=False)

+--------------------------------+------+-------------------+----------------------+--------+
|key_phrase                      |source|DocumentSimilarity |MMRScore              |sentence|
+--------------------------------+------+-------------------+----------------------+--------+
|one-week history polyuria       |ngrams|0.6088060363530704 |0.36528363632690924   |0       |
|HTG-induced pancreatitis years  |ngrams|0.584195512186101  |0.11729241005609656   |0       |
|insulin glargine night          |ngrams|0.5542673392385495 |0.0780034763234903    |0       |
|history gestational diabetes    |ngrams|0.5492484712733513 |0.07024012218392744   |0       |
|28-year-old female history      |ngrams|0.505395827262031  |0.12611559884148277   |0       |
|admitted starvation ketosis     |ngrams|0.5019899755932794 |0.042705157865605636  |0       |
|triglycerides 508 mg/dL         |ngrams|0.4961037608926256 |0.10460743161195774   |0       |
|vomiting weeks                  |ngrams|0.4524400777311215 

In [26]:
# sbert_jsl_medium_uncased (default)

ngram_ner_results.selectExpr("explode(key_phrases) AS key_phrase")\
                 .selectExpr(
                     "SUBSTRING(key_phrase.result, 0, 40) as key_phrase",
                     "IF(key_phrase.metadata.entity = 'UNK', 'ngrams', 'NER') AS source",
                     "key_phrase.metadata.DocumentSimilarity",
                     "key_phrase.metadata.MMRScore",
                     "key_phrase.metadata.sentence")\
                 .show(truncate=False)

+--------------------------------+------+-------------------+-------------------+--------+
|key_phrase                      |source|DocumentSimilarity |MMRScore           |sentence|
+--------------------------------+------+-------------------+-------------------+--------+
|type two diabetes mellitus      |NER   |0.7639752178409145 |0.4583851489191373 |0       |
|subsequent type diabetes        |ngrams|0.7503709550950611 |0.0829824679140303 |0       |
|HTG-induced pancreatitis years  |ngrams|0.6817062670747339 |0.11246277713293629|0       |
|hepatitis obesity               |ngrams|0.6666055227936242 |0.11770535787141301|0       |
|mellitus diagnosed years        |ngrams|0.6389213593287539 |0.0812948472932965 |0       |
|history gestational diabetes    |ngrams|0.6219875504416315 |0.09501042715667779|0       |
|vomiting                        |ngrams|0.5824236751787064 |0.14864177343228657|0       |
|admitted starvation ketosis     |ngrams|0.5789874663466789 |0.12008074933190521|0       |

**Lets change `.setConcatenateSentences`-> False for checking the results by using sentence level embeddings and getting their average for calculating the document average.**

In [27]:
ngram_ner_key_phrase_sent = ChunkKeyPhraseExtraction.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setTopN(10) \
    .setDivergence(0.4)\
    .setInputCols(["sentences", "merged_chunks"])\
    .setOutputCol("key_phrases")\
    .setConcatenateSentences(False)

ngram_ner_sent_pipeline = Pipeline(stages=[
    documenter, 
    sentencer, 
    tokenizer, 
    stop_words_cleaner,
    ngram_generator,
    embeddings, 
    ner_tagger, 
    ner_converter, 
    chunk_merger,
    ngram_ner_key_phrase_sent
])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [28]:
ngram_ner_sent_results = ngram_ner_sent_pipeline.fit(empty_data).transform(textDF)

In [29]:
# .setConcatenateSentences(False)

ngram_ner_sent_results.selectExpr("explode(key_phrases) AS key_phrase")\
                      .selectExpr(
                          "SUBSTRING(key_phrase.result, 0, 40) as key_phrase",
                          "IF(key_phrase.metadata.entity = 'UNK', 'ngrams', 'NER') AS source",
                          "key_phrase.metadata.DocumentSimilarity",
                          "key_phrase.metadata.MMRScore",
                          "key_phrase.metadata.sentence")\
                      .show(50,truncate=False)

+------------------------------------+------+--------------------+--------------------+--------+
|key_phrase                          |source|DocumentSimilarity  |MMRScore            |sentence|
+------------------------------------+------+--------------------+--------------------+--------+
|one-week history polyuria           |ngrams|0.15724817561087454 |0.09434890911561337 |0       |
|HTG-induced pancreatitis years      |ngrams|0.15313215735709926 |-0.14134561311865573|0       |
|insulin glargine night              |ngrams|0.13809328725786404 |-0.13992690658685297|0       |
|female history gestational          |ngrams|0.10183597128142986 |-0.11137554423918967|0       |
|HbA1c 10%                           |ngrams|0.09713646075486344 |-0.13530046808469487|0       |
|dapagliflozin T2DM atorvastatin     |ngrams|0.0923691803234179  |-0.09540792069419035|0       |
|days prior admission                |ngrams|0.08338123694760421 |-0.1567860458723211 |0       |
|33.5 kg/m2                   

In [ ]:
# .setConcatenateSentences(True) # default

ngram_ner_bio_results.selectExpr("explode(key_phrases) AS key_phrase")\
                     .selectExpr(
                         "SUBSTRING(key_phrase.result, 0, 40) as key_phrase",
                         "IF(key_phrase.metadata.entity = 'UNK', 'ngrams', 'NER') AS source",
                         "key_phrase.metadata.DocumentSimilarity",
                         "key_phrase.metadata.MMRScore",
                         "key_phrase.metadata.sentence")\
                     .show(truncate=False)

+--------------------------------+------+-------------------+----------------------+--------+
|key_phrase                      |source|DocumentSimilarity |MMRScore              |sentence|
+--------------------------------+------+-------------------+----------------------+--------+
|one-week history polyuria       |ngrams|0.6088060363530704 |0.36528363632690924   |0       |
|HTG-induced pancreatitis years  |ngrams|0.584195512186101  |0.11729241005609656   |0       |
|insulin glargine night          |ngrams|0.5542673392385495 |0.0780034763234903    |0       |
|history gestational diabetes    |ngrams|0.5492484712733513 |0.07024012218392744   |0       |
|28-year-old female history      |ngrams|0.505395827262031  |0.12611559884148277   |0       |
|admitted starvation ketosis     |ngrams|0.5019899755932794 |0.042705157865605636  |0       |
|triglycerides 508 mg/dL         |ngrams|0.4961037608926256 |0.10460743161195774   |0       |
|vomiting weeks                  |ngrams|0.4524400777311215 

# with YAKE Keyword Extraction
 
Lets get the keys phrases using `YakeKeywordExtraction` and compare the results with `ChunkKeyPhraseExtraction`.

In [30]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentencer = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("tokens") \
    .setSplitChars(['\[','\]']) \

stop_words_cleaner = StopWordsCleaner.pretrained()\
    .setInputCols("tokens")\
    .setOutputCol("clean_tokens")\
    .setCaseSensitive(False)

keywords = YakeKeywordExtraction() \
    .setInputCols("clean_tokens") \
    .setOutputCol("yake") \
    .setMinNGrams(1) \
    .setMaxNGrams(3)\
    .setNKeywords(20)\

yake_key_phrase_extractor = ChunkKeyPhraseExtraction.pretrained()\
    .setTopN(10) \
    .setDivergence(0.4)\
    .setInputCols(["sentences", "yake"])\
    .setOutputCol("yake_key_phrases")

yake_pipeline = Pipeline(stages=[
    documenter, 
    sentencer, 
    tokenizer, 
    stop_words_cleaner,
    keywords,
    yake_key_phrase_extractor
])

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
sbert_jsl_medium_uncased download started this may take some time.
Approximate size to download 146.8 MB
[OK!]


In [31]:
yake_results = yake_pipeline.fit(empty_data).transform(textDF)

**Lets check YAKE Keyword Extraction results and scores.**

In [32]:
yake_results.selectExpr("explode(yake) AS key_phrase_candidate").show(30,truncate=False)

+---------------------------------------------------------------------------------------------------+
|key_phrase_candidate                                                                               |
+---------------------------------------------------------------------------------------------------+
|{chunk, 91, 95, prior, {score -> 0.029673513395379065, sentence -> 0}, []}                         |
|{chunk, 100, 111, presentation, {score -> 0.03159661517236814, sentence -> 0}, []}                 |
|{chunk, 169, 173, prior, {score -> 0.029673513395379065, sentence -> 0}, []}                       |
|{chunk, 223, 227, prior, {score -> 0.029673513395379065, sentence -> 0}, []}                       |
|{chunk, 232, 243, presentation, {score -> 0.03159661517236814, sentence -> 0}, []}                 |
|{chunk, 445, 449, prior, {score -> 0.029673513395379065, sentence -> 0}, []}                       |
|{chunk, 454, 465, presentation, {score -> 0.03159661517236814, sentence -> 0}, []

In [33]:
scores = yake_results.selectExpr("explode(arrays_zip(yake.result, yake.metadata)) as resultTuples") \
                     .selectExpr("resultTuples['0'] as keyword", "resultTuples['1'].score as score")

In [34]:
scores.orderBy("score").show(20, truncate = False)

+------------------------------------+--------------------+
|keyword                             |score               |
+------------------------------------+--------------------+
|years prior presentation            |0.006335399690627251|
|years prior presentation            |0.006335399690627251|
|prior presentation                  |0.011644010991495998|
|prior presentation                  |0.011644010991495998|
|prior presentation                  |0.011644010991495998|
|weeks prior presentation            |0.020272229518351368|
|prior presentation subsequent       |0.020272229518351368|
|respiratory tract infection         |0.02568455658449274 |
|respiratory tract infection         |0.02568455658449274 |
|anion gap                           |0.025965846371439553|
|anion gap                           |0.025965846371439553|
|anion gap                           |0.025965846371439553|
|physical examination presentation   |0.02840600503736659 |
|obtained hours presentation         |0.

**Show top-10 results for YAKE.**

In [35]:
scores.select("keyword", "score").distinct().orderBy("score").show(truncate = False)

+------------------------------------+--------------------+
|keyword                             |score               |
+------------------------------------+--------------------+
|years prior presentation            |0.006335399690627251|
|prior presentation                  |0.011644010991495998|
|prior presentation subsequent       |0.020272229518351368|
|weeks prior presentation            |0.020272229518351368|
|respiratory tract infection         |0.02568455658449274 |
|anion gap                           |0.025965846371439553|
|physical examination presentation   |0.02840600503736659 |
|obtained hours presentation         |0.028532992974589392|
|examination presentation significant|0.028532992974589392|
|prior                               |0.029673513395379065|
|years prior                         |0.030808818777992058|
|anion gap elevated                  |0.031568192739369824|
|presentation                        |0.03159661517236814 |
|patient treated insulin             |0.

**Now we can compare the results with `ChunkKeyPhraseExtraction`.**

In [36]:
yake_results.select(F.explode(F.arrays_zip("yake_key_phrases.result","yake_key_phrases.metadata")).alias("cols"))\
            .select(F.expr("cols['0']").alias("key_phrase_candidate"),
                    F.expr("cols['1']['DocumentSimilarity']").alias("DocumentSimilarity"),
                    F.expr("cols['1']['MMRScore']").alias("MMRScore"),
                    F.expr("cols['1']['sentence']").alias("sentence")).show(truncate=False)

+------------------------------------+-------------------+---------------------+--------+
|key_phrase_candidate                |DocumentSimilarity |MMRScore             |sentence|
+------------------------------------+-------------------+---------------------+--------+
|pancreatitis years prior            |0.6491588635917195 |0.3894953336321851   |0       |
|diagnosed years prior               |0.38594468799232595|-0.03448213809873585 |0       |
|respiratory tract infection         |0.34452758398951444|-0.06206623419380611 |0       |
|patient treated insulin             |0.34134579194718073|-0.037567830250351125|0       |
|serum                               |0.3371023437059929 |0.024651741119600168 |0       |
|presentation revealed glucose       |0.31458362563143866|-0.11752329802552122 |0       |
|examination presentation significant|0.29099962673151175|-0.07343289069221776 |0       |
|prior analysis due                  |0.22501724451152066|-0.13367731084754253 |0       |
|prior    

**As you can see, `ChunkKeyPhraseExtraction` results are better than `YakeKeywordExtraction`.**